In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import talib
from sklearn.decomposition import PCA
import math
import plotly.express as px
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split



In [2]:

PATH = "../Data/DAX/yahoo_fin/"

def load(filename="1d"):
     data = pd.read_csv(PATH+"{}.csv".format(filename))
     return data
def save(filename, data):
     data.to_csv(PATH+filename+".csv")

def normalize(x):
    mean = np.mean(x, axis=0, keepdims=True)
    std = np.std(x, axis=0, keepdims=True)
    return (x - mean) / std


In [3]:
data = load("day")[33:]
print(data.columns)


Index(['Unnamed: 0', 'open', 'high', 'low', 'close', 'volume', 'intra-diff',
       'inter-diff', 'intra-derivative', 'inter-derivative', '1-rad', '1-err',
       '3-rad', '3-err', '5-rad', '5-err', '20-rad', '20-err', 'BB-upper',
       'BB-middle', 'BB-lower', 'ema5', 'ema20', 'sma50', 'sma200', 'sar',
       'adx', 'macd', 'macdsignal', 'macdhist', 'mom', 'rsi', 'stoch_k',
       'stoch_d', 'willr', 'bb-upper', 'bb-middle', 'bb-lower', 'ema-cross',
       'sma-cross', 'stoch-diff', 'sar-diff'],
      dtype='object')


In [25]:
def tf_model(period):
    feature_list = ["1-rad", "inter-derivative", 
                    "bb-upper", "bb-lower", "bb-middle", 
                    "ema-cross", "macdsignal", "macdhist", "macd", 
                    "rsi", "sar-diff", "stoch-diff", 
                    "intra-derivative", "1-err", "intra-diff"]
    
    N = len(feature_list)
    features = period[feature_list][:-1]
    features = features.values
    features = normalize(features)

    targets = period["inter-diff"][1:]
    targets = targets.values

    train_features, test_features, train_labels, test_labels = train_test_split(features, targets, test_size = 0.10, random_state = 2020)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(N),
        tf.keras.layers.Dense(2*N, activation='relu'),
        tf.keras.layers.Dense(N, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        # tf.keras.layers.Dense(N, activation='relu'),
        tf.keras.layers.Dense(N/2, activation='relu'),
        tf.keras.layers.Dense(1)
        ])

    model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])


    model.fit(train_features, train_labels, epochs=600, batch_size=100)
    model.evaluate(test_features,  test_labels, verbose=2)
    model.predict(test_features)


    
    

In [26]:
tf_model(data)

 0.3126
Epoch 423/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3109 - mean_absolute_error: 0.3109
Epoch 424/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3125 - mean_absolute_error: 0.3125
Epoch 425/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3075 - mean_absolute_error: 0.3075
Epoch 426/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3077 - mean_absolute_error: 0.3077
Epoch 427/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3119 - mean_absolute_error: 0.3119
Epoch 428/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3079 - mean_absolute_error: 0.3079
Epoch 429/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3095 - mean_absolute_error: 0.3095
Epoch 430/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3079 - mean_absolute_error: 0.3079
Epoch 431/600
59/59 [==============================] - 0s 2ms/step - loss: 0.3078 - mean_absolut